# Example for calculating the fn-coefficients

In the following, a simple example on how to manually evaluate the fn-coefficients is given. 
The ground is hereby defined as a Lambertian-surface and the covering layer is assumed to consist of Rayleigh-particles. Thus, we have: ($R_0$ hereby denotes the diffuse albedo of the surface)

Within the following calculation, the symbols and functions are defined as presented in the Theory-chapter of the documentation.

### Definition of the BRDF of the ground surface:

- $BRDF(\theta, \phi, \theta_{ex},\phi_{ex}) = \frac{R_0}{\pi}$

### Definition of the scattering phase-function of the covering layer:

- $p(\theta, \phi, \theta_{ex},\phi_{ex}) = \frac{3}{16\pi} (1+\cos(\Theta)^2) \quad$ with $\mbox{}\quad$ $\cos(\Theta) = \cos(\theta)\cos(\theta_{ex}) + \sin(\theta)\sin(\theta_{ex})\cos(\phi - \phi_{ex})$

### Manual Calculation of the fn-coefficients:

$INT := \int_0^{2\pi} p(\theta_0, \phi_0, \theta,\phi) * BRDF(\pi-\theta, \phi, \theta_{ex},\phi_{ex}) d\phi $

$\phantom{INT} = \frac{3 R_0}{16 \pi^2} \int_{0}^{2\pi}  (1+[\cos(\theta_0)\cos(\theta) + \sin(\theta_0)\sin(\theta)\cos(\phi_0 - \phi)]^2) d\phi$

$\phantom{INT} = \frac{3 R_0}{16 \pi^2} \int_0^{2\pi} (1+ \mu_0^2 \mu^2 + 2 \mu_0 \mu \sin(\theta_0) \sin(\theta) \cos(\phi_0 - \phi) + (1-\mu_0)^2(1-\mu)^2 \cos(\phi_0 - \phi)^2 d\phi$


where the shorthand-notation $\mu_x = \cos(\theta_x)$ has been introduced.
The above integral can now easily be solved by noticing:

$\int_0^{2\pi} \cos(\phi_0 - \phi)^n d\phi = \left\lbrace \begin{matrix} 2 \pi & \textrm{for } n=0 \\ 0 & \textrm{for } n=1 \\ \pi  & \textrm{for } n=2 \end{matrix} \right.$
    
Inserting the above solution and using some algebraic manipulations we therefore find:

$INT = \frac{3 R_0}{16\pi} \Big[ (3-\mu_0^2) + (3 \mu_0 -1) \mu^2 \Big] := \sum_{n=0}^2 f_n \, \mu^n $

$\Rightarrow \quad f_0 = \frac{3 R_0}{16\pi}(3-\mu_0^2) \qquad f_1 = 0 \qquad f_2 = \frac{3 R_0}{16\pi}(3 \mu_0 -1) \qquad f_n = 0 \, \forall \, n>2
$

## Evaluation of the fn-coefficients using the RT1-module

In [1]:
# imports
from rt1.rt1 import RT1 
from rt1.volume import Rayleigh   
from rt1.surface import Isotropic 


import sympy as sp
# enable printing sympy equations via latex-equation-rendering
sp.init_printing(use_latex='mathjax')

In [52]:
# definition of volume and surface
# (the values of tau and omega have no effect on the calculated fn-coefficients)
V = Rayleigh(tau=0.7, omega=0.3)

R0 = 1.    # set the diffuse reflectance
SRF = Isotropic(NormBRDF=R0)

In [53]:
I0 = 1.   # set incident intensity
R = RT1(I0, 0., 0., 0., 0., RV=V, SRF=SRF, geometry='mono')  
# setting geometry='vvvv' ensures that both incident- and scattering-angles 
# are treated symbolically

In [54]:
fn = R.fn #calculate fn-coefficients

# in order to see that the calculated coefficients are actually equal, we have
# to apply some trigonometric simplifications:

# apply basic trigonometric simplifications
fn = [sp.trigsimp(i) for i in R.fn]
# replace appearing sin^2 with 1-cos^2
fn = [i.xreplace({sp.sin('theta_0')**2 : 1.-sp.cos('theta_0')**2}) for i in fn] 
# convert floats to rationals and factor out
fn = [i.nsimplify().factor() for i in fn]

In [55]:
# print calculated fn-coefficients
fn

⎡   ⎛   2        ⎞        ⎛     2        ⎞      ⎤
⎢-3⋅⎝cos (θ₀) - 3⎠      3⋅⎝3⋅cos (θ₀) - 1⎠      ⎥
⎢──────────────────, 0, ──────────────────, 0, 0⎥
⎣       16⋅π                   16⋅π             ⎦